In [1]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme()
from scipy.stats import norm
from IPython.display import display

In [3]:
phiusiil_phishing_url_website_dataset = fetch_ucirepo(id=967)

X = phiusiil_phishing_url_website_dataset.data.features
y = phiusiil_phishing_url_website_dataset.data.targets

print(phiusiil_phishing_url_website_dataset.metadata)

# print(phiusiil_phishing_url_website_dataset.variables)

{'uci_id': 967, 'name': 'PhiUSIIL Phishing URL (Website) Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/967/data.csv', 'abstract': 'PhiUSIIL Phishing URL Dataset is a substantial dataset comprising 134,850 legitimate and 100,945 phishing URLs. Most of the URLs we analyzed, while constructing the dataset, are the latest URLs. Features are extracted from the source code of the webpage and URL. Features such as CharContinuationRate, URLTitleMatchScore, URLCharProb, and TLDLegitimateProb are derived from existing features.', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 235795, 'num_features': 54, 'feature_types': ['Real', 'Categorical', 'Integer'], 'demographics': [], 'target_col': ['label'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2024, 'last_updated': 'Thu

In [4]:
def resume_df(df):

  df.reset_index(drop=True,inplace=True)
  df_aux = pd.DataFrame(index=df.columns)
  df_aux["tipo"] = df.dtypes
  df_aux["%NaN"] = round(100*df.isna().sum()/df.shape[0],2)
  df_aux["uniques"] = df.nunique()
  df_aux["perc_valores_diff"] = round(100*df_aux["uniques"]/df.shape[0],2)

  return df_aux

In [5]:
resume = resume_df(X)
resume.loc[resume["perc_valores_diff"]<15.42]

,tipo,%NaN,uniques,perc_valores_diff
URLLength,int64,0.0,482,0.20
DomainLength,int64,0.0,101,0.04
IsDomainIP,int64,0.0,2,0.00
TLD,object,0.0,695,0.29
CharContinuationRate,float64,0.0,898,0.38
TLDLegitimateProb,float64,0.0,465,0.20
TLDLength,int64,0.0,12,0.01
NoOfSubDomain,int64,0.0,10,0.00
HasObfuscation,int64,0.0,2,0.00
NoOfObfuscatedChar,int64,0.0,20,0.01


Proporção entre as classes estudadas para resolução do problema

In [10]:
round(y.value_counts(normalize=False),4)

label
1        134850
0        100945
dtype: int64

Inicialmente, 5 features (atributos) serão realizados para compararmos a distribuição entre as classes:

- NoOfPopup
- NoOfImage
- NoOfCSS
- NoOfJS
- NoOfExternalRef

In [7]:
df = pd.concat([X,y],axis=1).copy()

In [8]:


display(df.loc[df['label']==0,"NoOfPopup"].describe())


count    100945.000000
mean          0.009758
std           0.173007
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          10.000000
Name: NoOfPopup, dtype: float64

In [9]:
def compare_describe(data0,data1):

    describe_0 = data0.describe()
    describe_1 = data1.describe()

    compare_df = pd.concat([describe_0,describe_1], axis=1)
    compare_df.columns = ['Classe_0', 'Classe_1']

    return compare_df

for c in ['NoOfPopup','NoOfImage','NoOfCSS','NoOfJS','NoOfExternalRef']:
  print(f"Descritiva da Feature:",c)
  print("-------------------------------------------")
  display(compare_describe(df.loc[df['label']==0,c],df.loc[df['label']==1,c]))
  print("-------------------------------------------")

Descritiva da Feature: NoOfPopup
-------------------------------------------


,Classe_0,Classe_1
count,100945.000000,134850.000000
mean,0.009758,0.380467
std,0.173007,5.110220
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,10.000000,602.000000


-------------------------------------------
Descritiva da Feature: NoOfImage
-------------------------------------------


,Classe_0,Classe_1
count,100945.000000,134850.000000
mean,0.866492,44.946600
std,3.296497,100.930539
min,0.000000,0.000000
25%,0.000000,13.000000
50%,0.000000,25.000000
75%,0.000000,47.000000
max,158.000000,8956.000000


-------------------------------------------
Descritiva da Feature: NoOfCSS
-------------------------------------------


...

-------------------------------------------
Descritiva da Feature: NoOfJS
-------------------------------------------


,Classe_0,Classe_1
count,100945.000000,134850.000000
mean,0.890336,17.732532
std,3.362975,27.213939
min,0.000000,0.000000
25%,0.000000,8.000000
50%,0.000000,14.000000
75%,0.000000,23.000000
max,102.000000,6957.000000


-------------------------------------------
Descritiva da Feature: NoOfExternalRef
-------------------------------------------


,Classe_0,Classe_1
count,100945.000000,134850.000000
mean,1.128119,85.294601
std,3.204587,205.669475
min,0.000000,0.000000
25%,0.000000,17.000000
50%,0.000000,46.000000
75%,1.000000,106.000000
max,368.000000,27516.000000


-------------------------------------------
